<a href="https://colab.research.google.com/github/nisaral/DL_practice_grounds/blob/main/TabularToGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Columsn for Node Representation
* Node Types:
    *	Drugs
    *	Proteins/Targets
    *	Diseases

    - Columns considered for Drug nodes
        *	DRUGID
        *	DRUGTYPE
        *	Drug_Status
        *	Drug_high_status

    - columns considered for Protein or Target nodes
        *	TargetID
        *	UNIPROID
        *	BIOCLASS
        *	SEQUENCE
        *	Target_Status

    - columns considered for Disease Nodes
        *	Disease_of_highest_status
        *	Accession Number

2.	Edge Representation
*  	Edge Types consideration as initial scope:
    *   Drug-Target Interactions
    *   Drug-Disease Associations

Columns for Defining Edges and their Strength:
a. Defining Edges:
*	Drug-Target Edges:
    *	Source Node: Drug (DRUGID)
    *	Target Node: protein or target (TargetID or UNIPROID)

*	Drug-Disease Edges:
    *	Source Node: Drug (DRUGID)
    *	Target Node: Disease (Disease_of_highest_status or Accession Number)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("./FINAL_DRUG_REP.csv")
main_df = df.sample(frac=0.01)

FileNotFoundError: [Errno 2] No such file or directory: './FINAL_DRUG_REP.csv'

In [ ]:
main_df.head()

NameError: name 'main_df' is not defined

In [ ]:
drug_nodes = ['DRUGID', 'DRUGTYPE', 'Drug_Status', 'Drug_high_status']
target_nodes = ['TargetID', 'UNIPROID', 'BIOCLASS', 'SEQUENCE', 'Target_Status']
disease_nodes = ['Disease_of_highest_status', 'Accession Number']

In [ ]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1921 entries, 116905 to 120333
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   TargetID                   1921 non-null   object
 1   DRUGID                     1921 non-null   object
 2   DRUGTYPE                   1921 non-null   object
 3   Drug_high_status           1921 non-null   object
 4   DRUGNAME                   1921 non-null   object
 5   PUBCHCID                   1921 non-null   int64 
 6   Disease_of_highest_status  1921 non-null   object
 7   Drug_Status                1921 non-null   object
 8   UNIPROID                   1921 non-null   object
 9   TARGNAME                   1921 non-null   object
 10  GENENAME                   1921 non-null   object
 11  SYNONYMS                   1921 non-null   object
 12  FUNCTION                   1921 non-null   object
 13  BIOCLASS                   1921 non-null   object
 14  SEQUEN

In [ ]:
main_df.shape

(1921, 18)

In [ ]:
drug_df = main_df[drug_nodes]

In [ ]:
drug_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1921 entries, 116905 to 120333
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DRUGID            1921 non-null   object
 1   DRUGTYPE          1921 non-null   object
 2   Drug_Status       1921 non-null   object
 3   Drug_high_status  1921 non-null   object
dtypes: object(4)
memory usage: 75.0+ KB


In [ ]:
drug_df.nunique()

,0
DRUGID,872
DRUGTYPE,3
Drug_Status,22
Drug_high_status,24


In [ ]:
drug_df.head()

,DRUGID,DRUGTYPE,Drug_Status,Drug_high_status
116905,D0T6QX,Small molecular drug,Approved,Approved
15681,D0E6UZ,Small molecular drug,Phase 1,Phase 1/2
71766,D04QWQ,Small molecular drug,Phase 2/3,Phase 2/3
80034,D0M6VK,Small molecular drug,Approved,Approved
22379,D01DVX,Small molecular drug,Terminated,Terminated


In [ ]:
drug_df = drug_df.drop_duplicates()
drug_df.shape

(987, 4)

In [ ]:
drug_df.head()

,DRUGID,DRUGTYPE,Drug_Status,Drug_high_status
116905,D0T6QX,Small molecular drug,Approved,Approved
15681,D0E6UZ,Small molecular drug,Phase 1,Phase 1/2
71766,D04QWQ,Small molecular drug,Phase 2/3,Phase 2/3
80034,D0M6VK,Small molecular drug,Approved,Approved
22379,D01DVX,Small molecular drug,Terminated,Terminated


In [ ]:
drug_df.nunique()

,0
DRUGID,872
DRUGTYPE,3
Drug_Status,22
Drug_high_status,24


In [ ]:
drug_id_counts = drug_df['DRUGID'].value_counts()

drug_id_counts[drug_id_counts>1].shape

(94,)

In [ ]:
# dropped since cardinality of DRUGID column
# and total number of rows is similar. Is this right?

drug_df = drug_df.drop('DRUGID', axis=1)
drug_df.head()

,DRUGTYPE,Drug_Status,Drug_high_status
116905,Small molecular drug,Approved,Approved
15681,Small molecular drug,Phase 1,Phase 1/2
71766,Small molecular drug,Phase 2/3,Phase 2/3
80034,Small molecular drug,Approved,Approved
22379,Small molecular drug,Terminated,Terminated


In [ ]:
drug_id_mapping = drug_df.index
drug_df = drug_df.reset_index()


drug_node_features = pd.concat([
    pd.get_dummies(drug_df['DRUGTYPE']),
    pd.get_dummies(drug_df['Drug_Status']),
    pd.get_dummies(drug_df['Drug_high_status'])
], axis=1)

drug_node_features = drug_node_features.astype(int)

drug_node_features.head()

,Combination drug (Antibody),Small molecular drug,Small molecule immunotherapy,Application submitted,Approved,Clinical trial,Discontinued in Phase 1,Discontinued in Phase 1/2,Discontinued in Phase 2,Discontinued in Phase 2b,...,Phase 2,Phase 2 Trial,Phase 2/3,Phase 2a,Phase 3,Phase 4,Preclinical,Registered,Terminated,Withdrawn from market
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
X_drug = drug_node_features.to_numpy()
X_drug.shape

(987, 49)

In [ ]:
from collections import Counter
def cumulatively_categorise(column,threshold=0.65,return_categories_list=True, filler='Other'):
  threshold_value=int(threshold*len(column))
  categories_list=[]
  s=0
  counts=Counter(column)

  for i,j in counts.most_common():
    s+=dict(counts)[i]
    categories_list.append(i)

    if s>=threshold_value:
      break
  categories_list.append(filler)

  new_column=column.apply(lambda x: x if x in categories_list else filler)

  if(return_categories_list):
    return new_column,categories_list
  else:
    return new_column

In [ ]:
target_df = main_df[target_nodes]

In [ ]:
target_df.nunique()

,0
TargetID,331
UNIPROID,330
BIOCLASS,64
SEQUENCE,328
Target_Status,23


In [ ]:
target_df.head()

,TargetID,UNIPROID,BIOCLASS,SEQUENCE,Target_Status
116905,T83813,5HT2C_HUMAN,GPCR rhodopsin,MVNLRNAVHSFLVHLIGLLVWQCDISVSPVAAIVTDIFNTSDGGRF...,Phase 2
15681,T95913,PK3CG_HUMAN,Kinase,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...,Phase 1
71766,T23172,JAK3_HUMAN,Kinase,MAPPSEETPLIPQRSCSLLSTEAGALHVLLPARGPGPPQRLSFSFG...,Phase 2
80034,T28893,ACM1_HUMAN,GPCR rhodopsin,MNTSAPPAVSPNITVLAPGKGPWQVAFIGITTGLLSLATVTGNLLV...,Phase 2
22379,T34296,MMP13_HUMAN,Peptidase,MHPGVLAAFLFLSWTHCRALPLPSGGDEDDLSEEDLQFAERYLRSY...,Discontinued in Phase 2


In [ ]:
target_df.shape

(1921, 5)

In [ ]:
target_df = target_df.drop_duplicates()
target_df.shape

(843, 5)

In [ ]:
# dropped UNIPROID due to similar cardinality with TargetID.

target_df = target_df.drop('UNIPROID', axis=1)
target_df.head()

,TargetID,BIOCLASS,SEQUENCE,Target_Status
116905,T83813,GPCR rhodopsin,MVNLRNAVHSFLVHLIGLLVWQCDISVSPVAAIVTDIFNTSDGGRF...,Phase 2
15681,T95913,Kinase,MELENYKQPVVLREDNCRRRRRMKPRSAAASLSSMELIPIEFVLPT...,Phase 1
71766,T23172,Kinase,MAPPSEETPLIPQRSCSLLSTEAGALHVLLPARGPGPPQRLSFSFG...,Phase 2
80034,T28893,GPCR rhodopsin,MNTSAPPAVSPNITVLAPGKGPWQVAFIGITTGLLSLATVTGNLLV...,Phase 2
22379,T34296,Peptidase,MHPGVLAAFLFLSWTHCRALPLPSGGDEDDLSEEDLQFAERYLRSY...,Discontinued in Phase 2


In [ ]:
target_df['TargetID_filtered'] = cumulatively_categorise(column=target_df['TargetID'], return_categories_list=False, filler='RareTargetID')

In [ ]:
target_df['SEQUENCE_filtered'] = cumulatively_categorise(column=target_df['TargetID'], return_categories_list=False, filler='RareSeq')

In [ ]:
target_df.drop(['TargetID','SEQUENCE'], axis=1, inplace=True)

In [ ]:
target_id_mapping = target_df.index
target_df = target_df.reset_index()


target_node_features = pd.concat([
    pd.get_dummies(target_df['TargetID_filtered']),
    pd.get_dummies(target_df['SEQUENCE_filtered']),
    pd.get_dummies(target_df['BIOCLASS']),
    pd.get_dummies(target_df['Target_Status']),
], axis=1)

target_node_features = target_node_features.astype(int)

target_node_features.head()

,RareTargetID,T00140,T00884,T01777,T02551,T03691,T05031,T06093,T06273,T07303,...,Phase 2,Phase 2/3,Phase 2a,Phase 3,Phase 4,Preclinical,Preregistration,Registered,Terminated,Withdrawn from market
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_target = target_node_features.to_numpy()
X_target.shape

(843, 319)

In [ ]:
disease_df = main_df[disease_nodes]
disease_df.nunique()

,0
Disease_of_highest_status,347
Accession Number,326


In [ ]:
disease_df.shape

(1921, 2)

In [ ]:
disease_df.head()

,Disease_of_highest_status,Accession Number
116905,Anxiety disorder,P28335
15681,Diffuse large B-cell lymphoma,P48736
71766,Rheumatoid arthritis,P52333
80034,Peptic ulcer,P11229
22379,Rheumatoid arthritis,P45452


In [ ]:
disease_df = disease_df.drop_duplicates()
disease_df.shape

(1398, 2)

In [ ]:
disease_df.nunique()

,0
Disease_of_highest_status,347
Accession Number,326


In [ ]:
disease_df.head()

,Disease_of_highest_status,Accession Number
116905,Anxiety disorder,P28335
15681,Diffuse large B-cell lymphoma,P48736
71766,Rheumatoid arthritis,P52333
80034,Peptic ulcer,P11229
22379,Rheumatoid arthritis,P45452


In [ ]:
disease_df['Disease_of_highest_status_filtered'], new_categories = cumulatively_categorise(column=disease_df['Disease_of_highest_status'], return_categories_list=True)

In [ ]:
disease_df.nunique()

,0
Disease_of_highest_status,347
Accession Number,326
Disease_of_highest_status_filtered,75


In [ ]:
disease_df['Accession Number'].value_counts().to_dict()

{'P00533': 40,
 'P14416': 32,
 'Q12809': 32,
 'P08908': 24,
 'P08684': 22,
 'P48736': 22,
 'P35968': 20,
 'Q13547': 19,
 'P28223': 19,
 'Q14247': 19,
 'P00918': 19,
 'P35372': 18,
 'P27338': 17,
 'P36888': 16,
 'P35367': 15,
 'Q99720': 14,
 'P11229': 14,
 'P15121': 14,
 'Q86W50': 13,
 'P03372': 13,
 'P11362': 13,
 'P31645': 13,
 'P00915': 12,
 'P10275': 12,
 'P10721': 12,
 'P29274': 11,
 'P08912': 11,
 'P21397': 11,
 'P42345': 11,
 'P23975': 11,
 'P41143': 11,
 'P35354': 11,
 'P31749': 11,
 'P52333': 11,
 'Q14524': 11,
 'P10635': 10,
 'P00734': 10,
 'P49841': 10,
 'P28335': 9,
 'P08183': 9,
 'P23219': 9,
 'P42336': 9,
 'P09960': 9,
 'O60674': 9,
 'O00329': 9,
 'P07333': 9,
 'P06213': 9,
 'Q92731': 9,
 'P42338': 8,
 'P21728': 8,
 'P29275': 8,
 'P16234': 8,
 'P41145': 8,
 'P08246': 8,
 'P22303': 8,
 'P37288': 7,
 'P35462': 7,
 'P24941': 7,
 'P41595': 7,
 'P0DMS8': 7,
 'P35348': 7,
 'P00519': 7,
 'P09917': 7,
 'P30542': 7,
 'Q8NER1': 7,
 'P25101': 6,
 'P50750': 6,
 'P18825': 6,
 'Q8WXI2':

In [ ]:
disease_df['Accession Number_filtered'] = cumulatively_categorise(column=disease_df['Accession Number'], return_categories_list=False)

In [ ]:
disease_df.nunique()

,0
Disease_of_highest_status,347
Accession Number,326
Disease_of_highest_status_filtered,75
Accession Number_filtered,82


In [ ]:
disease_df.drop(['Accession Number', 'Disease_of_highest_status'], axis=1, inplace=True)

In [ ]:
disease_id_mapping = disease_df.index
disease_df = disease_df.reset_index()


disease_node_features = pd.concat([
    pd.get_dummies(disease_df['Accession Number_filtered']),
    pd.get_dummies(disease_df['Disease_of_highest_status_filtered'])
], axis=1)

disease_node_features = disease_node_features.astype(int)

disease_node_features.head()

,O00329,O14965,O60674,Other,P00519,P00533,P00734,P00747,P00915,P00918,...,Pulmonary arterial hypertension,Renal cell carcinoma,Rheumatoid arthritis,Schizophrenia,Solid tumour/cancer,Squamous head and neck cell carcinom,Systemic mastocytosis,Thrombocytopenia,Thrombosis,Type-2 diabetes
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
X_disease = disease_node_features.to_numpy()
X_disease.shape

(1398, 157)

In [ ]:
len(drug_id_mapping)

987

In [ ]:
len(target_id_mapping)

843

In [ ]:
len(disease_id_mapping)

1398

In [ ]:
import torch
from torch_geometric.data import HeteroData

In [ ]:
import numpy as np

list1 = drug_id_mapping
list2 = target_id_mapping
list3 = disease_id_mapping

# Convert the lists to sets for efficient intersection
set_A = set(list1)
set_B = set(list2)
set_C = set(list3)

# Create edges based on shared indices
edges_AB = np.array([[i, j] for i in list1 for j in list2 if i == j]).T  # Edges between Type A and B
edges_AC = np.array([[i, j] for i in list1 for j in list3 if i == j]).T  # Edges between Type A and C
edges_BC = np.array([[i, j] for i in list2 for j in list3 if i == j]).T  # Edges between Type B and C

# Convert edge lists to PyTorch tensors
edges_AB = torch.tensor(edges_AB, dtype=torch.long) if edges_AB.size else torch.empty((2, 0), dtype=torch.long)
edges_AC = torch.tensor(edges_AC, dtype=torch.long) if edges_AC.size else torch.empty((2, 0), dtype=torch.long)
edges_BC = torch.tensor(edges_BC, dtype=torch.long) if edges_BC.size else torch.empty((2, 0), dtype=torch.long)

data = HeteroData()

# Add node types and their feature matrices
data['A'].x = drug_node_features
data['B'].x = target_node_features
data['C'].x = disease_node_features

# Add edges between different node types
data['A', 'to', 'B'].edge_index = edges_AB
data['A', 'to', 'C'].edge_index = edges_AC
data['B', 'to', 'C'].edge_index = edges_BC

